#### Import Libraries

In [ ]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# Data Understanding

In [ ]:
dataset_path = 'Dataset'
subsets = ['Training','Testing'] # Dataset divided into 2 subsets: training and testing

In [ ]:
image_data = []

for subset in subsets: # goes through the 2 subsets training and testing
    subset_path = os.path.join(dataset_path,subset)
    diagnoses = os.listdir(subset_path) # Returns a list of diagnoses

    for diag in diagnoses: # iterates over the diagnoses
        diagnoses_folder = os.path.join(subset_path, diag )
        for img_file in os.listdir(diagnoses_folder):
            img_path = os.path.join(diagnoses_folder, img_file) # gets the image path within the diagnoses folder 
            with Image.open(img_path) as img: 
                width, height = img.size # Collects the width, height, and mode of the images
                mode = img.mode
                image_data.append({ # adds all the collected info for each image to the list
                    'subset':subset,
                    'diagnosis': diag,
                    'width': width,
                    'height': height,
                    'mode': mode,
                    'image': np.array(img) #image converted to numpy array
                })


df = pd.DataFrame(image_data)
    

In [ ]:
df.head()

In [ ]:
# Number of images for training and testing for each diagnosis
table = df.groupby(['subset', 'diagnosis']).size().unstack()
table

In [ ]:
df_train = df[df['subset'] == 'Training']
df_test = df[df['subset'] == 'Testing']

In [ ]:
training_data = table.loc['Training'] 
testing_data= table.loc['Testing']

fig, ax = plt.subplots(ncols = 3,figsize=(20,14))

# training data split
ax[0].pie(training_data, labels= training_data.index,colors = ["#F9ED69", "#F08A5D", "#B83B5E", "#6A2C70"], 
    autopct=lambda p: '{:.2f}%\n{:,.0f}'.format(p, p * sum(df_train['diagnosis'].value_counts()) / 100),
    textprops={'fontsize': 20})
ax[0].set_title("Training")

# train test split
ax[1].set_title("Train Test Split")
ax[1].pie(
    [len(df_train), len(df_test)], 
    labels=['Train', 'Test'], 
    colors=['#144272', 'orange'],  # Fixed colors syntax and spelling
    autopct=lambda p: '{:.2f}%\n{:,.0f}'.format(p, p * (len(df_train) + len(df_test)) / 100),
    startangle=85, 
    textprops={'fontsize': 20}
)

# testing data split
ax[2].set_title('Testing Data')
ax[2].pie(testing_data, labels= testing_data.index, colors=["#F9ED69", "#F08A5D", "#B83B5E", "#6A2C70"], 
    autopct=lambda p: '{:.2f}%\n{:,.0f}'.format(p, p * sum(df_test['diagnosis'].value_counts()) / 100),
    textprops={'fontsize': 20 })

plt.show()

In [ ]:
# Distribution of image dimensions 
plt.figure(figsize=(8,5))
sns.scatterplot(data = df, x='width',y='height', hue='diagnosis',alpha=0.6)
plt.title('Image dimensions by diagnosis')
plt.xlabel('Width')
plt.ylabel('Height')
plt.legend(title='Diagnosis')
plt.show()

In [ ]:
# Displaying the mode of images, shows inconsistency across images
df['mode'].value_counts()

In [ ]:
# The number
sns.countplot(data=df, x='mode', hue='diagnosis', palette=['royalblue', 'peru', 'red', 'green'],legend='full')
plt.title('The number of images per image mode')
plt.xlabel('Mode')
plt.ylabel('Number of images')
plt.legend(title='Mode')
plt.show()

In [ ]:
# Take 5 images from each diagnosis and visualise them, raw images without processing
for diag in diagnoses:
    samples = [item for item in image_data if item['diagnosis'] == diag]
    samples = random.sample(samples, k=5)  
    
    plt.figure(figsize=(15, 3))
    for i, item in enumerate(samples):
        plt.subplot(1, 5, i + 1)
        plt.imshow(item['image'])
        plt.axis('off')
    plt.suptitle(f"Diagnosis: {diag}",fontsize=20)
    plt.show()
    


## Findings
The brain tumors dataset consists of 4 different brain diagnoses: Glioma, meningioma, pituitary and healthy (known as notumor). There are 7023 images; 5645 for training (80%) and 1411 for testing (20%) divided closely between the four diagnoses, also each subset is well divided among the 4 classes with a maximum of 28.35% for healthy brain and a minimum of 22.89% for glioma in the training set, with nearly similar results for the testing set indicating a good class balance to start with.

However, the image dimensions by diagnosis chart shows a huge variance in image dimensions that needs to be handled as part of the data preparation, we need to choose a common image size for all the images which may affect the performance of the model as certain parts of the brain may/may not be taken when resizing especially when it comes to locating tumors within the brain. Also the fact that some images have different color modes (3926 RGB and 3093 Greyscale) so we need to ensure consistency among shapes and channels to avoid potential issues.

# Data Preparation

In [ ]:
train_dir = 'Dataset/Training' # train images directory
test_dir = 'Dataset/Testing'  # test images directory
batch_size= 32 # define batch size
image_size = 224 # define image size

In [ ]:
#load training data using image_dataset_from_directory
#source: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,                                     
    label_mode= 'int',                             # Labels are encoded as a integers
    validation_split= 0.2,                         # 20% of the training data will be used for training
    color_mode = 'rgb',                            # Load all images in RGB mode 
    batch_size = batch_size,                       # number of images processed in each batch
    image_size = (image_size,image_size),          # image size is 224x224 (this will fix the imbalance we plotted earlier by using a uniform size)
    seed = 111,
    subset = 'training'                            # Training subset
)

#load validation data using image_dataset_from_directory
val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    label_mode= 'int',
    validation_split= 0.2,
    color_mode = 'rgb',
    batch_size = batch_size,
    image_size = (image_size,image_size),
    shuffle = False,                                # Validation data isn't shuffled
    seed = 111,
    subset = 'validation'                           # Validation subset
)

#load validation data using image_dataset_from_directory
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    label_mode= 'int',
    color_mode = 'rgb',
    batch_size = batch_size,
    image_size = (image_size, image_size)
)

In [ ]:
# Data augmentation
# Source: https://www.tensorflow.org/api_docs/python/tf/keras/layers
data_augmentation = Sequential([
    layers.RandomFlip('horizontal'),                                                             # Randomly flips images horizontally
    layers.RandomRotation(0.03,fill_mode='constant'),                                            # Randomly rotates images by ±10 degrees
    layers.RandomContrast(0.1),                                                                  # Randomly adjusts contrast by ±10%
    layers.RandomZoom(height_factor=0.01, width_factor=0.05),                                    # Randomly zooms in/out up to 1% vertically and 5% horizontally
    layers.RandomTranslation(height_factor=0.0015, width_factor=0.0015, fill_mode='constant'),   # Randomly shifts the image by 0.15% of the height and width
   
])

rescale = layers.Rescaling(1./255)  # Rescale pixel values to [0, 1] range

In [ ]:
# Source: https://www.tensorflow.org/tutorials/images/data_augmentation

AUTOTUNE = tf.data.AUTOTUNE # Allows TensorFlow to automatically choose the optimal number of parallel threads to improve performance

# training data augmented and rescaled (Images were already resized when loaded so we don't need to resize again)
train_ds = train_ds.map(
    lambda x, y: (rescale(data_augmentation(x,training= True)), y),
    num_parallel_calls= AUTOTUNE
)

# Validation data rescaled only
val_ds = val_ds.map(
    lambda x, y: (rescale(x), y),
    num_parallel_calls= AUTOTUNE
)

# Testing data rescaled only
test_ds = test_ds.map(
    lambda x, y: (rescale(x), y),
    num_parallel_calls= AUTOTUNE
)

# Allows later elements to be prepared while the current element is being processed which improves performance
train_ds_preprocessed = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds_preprocessed = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds_preprocessed = test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
def visualize_augmentation(data, diagnoses, num_samples):
    
    # empty dictionary that will contain an image per diagnosis
    diagnoses_images = {diag: None for diag in range(len(diagnoses))}
    
    # Unbatch data and get individual images
    for image, label in data.unbatch().shuffle(1000):
        label = int(label.numpy()) # convert tensorflow label to a python integer
        if diagnoses_images[label] is None:  # If we didn't store an image
            img = (image.numpy() * 255).astype(np.uint8) # convert image to an array and we scale it back to 0-255 (img_size = 224)
            diagnoses_images[label] = img # we add the image to the dictionary
       
            
    
    # Create visualization grid
    num_diagnoses = len(diagnoses) 
    fig, axes = plt.subplots(
        num_diagnoses,
        num_samples + 1,
        figsize=(2.5*(num_samples+1), 2.5*num_diagnoses))
    
    for class_idx in range(num_diagnoses):
        # Original image taken earlier
        orig_img = diagnoses_images[class_idx]
        # Add it in the first column of each row 
        axes[class_idx, 0].imshow(orig_img)
        axes[class_idx, 0].set_title(f"Original\n{diagnoses[class_idx]}")
        axes[class_idx, 0].axis('off')
        
        for aug_idx in range(1, num_samples+1):
            # We augment the original images 
            aug_img = data_augmentation(tf.expand_dims(orig_img, 0))
            aug_img = aug_img[0].numpy().astype('uint8')

            # then we add it to the visualisation grid
            axes[class_idx, aug_idx].imshow(aug_img)
            axes[class_idx, aug_idx].set_title(f"Augmented image {aug_idx}")
            axes[class_idx, aug_idx].axis('off')
    
    plt.tight_layout()
    plt.show()


In [ ]:
visualize_augmentation(
    data=train_ds,
    diagnoses = diagnoses,  
    num_samples=5
)

# Modelling

In [ ]:
input_shape = (224,224,3)
num_classes= 4

In [ ]:
def build_minimal_cnn():
    model = Sequential([
        Conv2D(16, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2,2)),
        
        Conv2D(32, (3,3), activation='relu'),
        MaxPooling2D((2,2)),

        Flatten(),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Build & train briefly for a small number of epochs (e.g., 5)
model_basic = build_minimal_cnn()
history_v1 = model_basic.fit(
    train_ds_preprocessed, 
    validation_data=val_ds_preprocessed, 
    epochs=10
)

# Evaluate quickly on validation
val_loss_v1, val_acc_v1 = model_basic.evaluate(test_ds_preprocessed)
print(f"Testing Accuracy: {val_acc_v1:.4f}")
